In [94]:
try:
    !pip install pyspark=="2.4.5"  --quiet
except:
 print("Running throw py file.")

In [75]:
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkFiles
from pyspark.sql.types import StringType
import pyspark

In [5]:
spark = SparkSession\
        .builder\
        .appName("Estudo Spark UI - Fabio Kfouri")\
        .getOrCreate()

## Leitura de dados usando uma fonte pública

Fonte de Dados:

https://data.humdata.org/dataset/faostat-prices-for-brazil

In [6]:
spark.sparkContext.addFile('https://data.humdata.org/dataset/bdf7bcca-28ae-47c5-8993-c87a7c5c04c0/resource/c16c15f5-efaf-4950-b848-94935987d312/download/producer-prices_bra.csv')

In [64]:
df = spark.read.csv(SparkFiles.get("producer-prices_bra.csv"), header = True, sep = ",")

In [65]:
df.show(5)

+-------------+-----------+----------+---------+-------------+---------------+---------------+------------+--------------------+---------+----------+-----------+------------+---------------+--------------------+----+
|         Iso3|  StartDate|   EndDate|Area Code|         Area|      Item Code|           Item|Element Code|             Element|Year Code|      Year|Months Code|      Months|           Unit|               Value|Flag|
+-------------+-----------+----------+---------+-------------+---------------+---------------+------------+--------------------+---------+----------+-----------+------------+---------------+--------------------+----+
|#country+code|#date+start| #date+end|     null|#country+name|#indicator+code|#indicator+name|        null|                null|     null|#date+year|       null|        null|#indicator+type|#indicator+value+num|null|
|          BRA| 1991-01-01|1991-12-31|       21|       Brazil|            515|         Apples|        5530|Producer Price (L...|    

In [89]:
df.describe()

DataFrame[summary: string, Iso3: string, StartDate: string, EndDate: string, Area Code: string, Area: string, Item Code: string, Item: string, Element Code: string, Element: string, Year Code: string, Year: string, Months Code: string, Months: string, Unit: string, Value: string, Flag: string]

In [90]:
df.printSchema()

root
 |-- Iso3: string (nullable = true)
 |-- StartDate: string (nullable = true)
 |-- EndDate: string (nullable = true)
 |-- Area Code: string (nullable = true)
 |-- Area: string (nullable = true)
 |-- Item Code: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Element Code: string (nullable = true)
 |-- Element: string (nullable = true)
 |-- Year Code: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Months Code: string (nullable = true)
 |-- Months: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Flag: string (nullable = true)



In [91]:
#df =
df = df.filter("Iso3 <> '#country+code'")\
    .withColumn('Value',df["Value"].cast('float'))\
    .withColumn('StartDate', F.to_date('StartDate', 'yyyy-MM-dd'))\
    .withColumn('EndDate', F.to_date('EndDate', 'yyyy-MM-dd'))

In [92]:
df.show(5, False)

+----+----------+----------+---------+------+---------+------+------------+--------------------------+---------+----+-----------+------------+----+---------+----+
|Iso3|StartDate |EndDate   |Area Code|Area  |Item Code|Item  |Element Code|Element                   |Year Code|Year|Months Code|Months      |Unit|Value    |Flag|
+----+----------+----------+---------+------+---------+------+------------+--------------------------+---------+----+-----------+------------+----+---------+----+
|BRA |1991-01-01|1991-12-31|21       |Brazil|515      |Apples|5530        |Producer Price (LCU/tonne)|1991     |1991|7021       |Annual value|LCU |105691.0 |*   |
|BRA |1992-01-01|1992-12-31|21       |Brazil|515      |Apples|5530        |Producer Price (LCU/tonne)|1992     |1992|7021       |Annual value|LCU |1969620.0|*   |
|BRA |1993-01-01|1993-12-31|21       |Brazil|515      |Apples|5530        |Producer Price (LCU/tonne)|1993     |1993|7021       |Annual value|LCU |26770.0  |*   |
|BRA |1994-01-01|1994-

In [93]:
df.printSchema()

root
 |-- Iso3: string (nullable = true)
 |-- StartDate: date (nullable = true)
 |-- EndDate: date (nullable = true)
 |-- Area Code: string (nullable = true)
 |-- Area: string (nullable = true)
 |-- Item Code: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Element Code: string (nullable = true)
 |-- Element: string (nullable = true)
 |-- Year Code: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Months Code: string (nullable = true)
 |-- Months: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Value: float (nullable = true)
 |-- Flag: string (nullable = true)



## Spark Catalog

In [23]:
spark.catalog.listTables()

[]

Cria View e exibe todas as listas

In [37]:
df.createOrReplaceTempView("temp")
spark.catalog.listTables()

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [38]:
spark.catalog.listTables()

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

Verifica se a tabela/view esta em cache

In [39]:
spark.catalog.isCached('temp')

False

Cria cache

In [40]:
spark.catalog.cacheTable('temp')
spark.catalog.isCached('temp')

True

Remove o Cache

In [41]:
spark.catalog.uncacheTable('temp')
spark.catalog.isCached('temp')

False

Limpa todos os caches

In [42]:
spark.catalog.cacheTable('temp')
spark.catalog.clearCache()
spark.catalog.isCached('temp')

False

Remove a View temp e exibe as existentes  

In [45]:
spark.catalog.dropTempView('temp')
spark.catalog.listTables()

[]